# UFO's <a id='H-UFOs'></a>
## Introductie van de notebook 

In dit notebook wordt een dataset over ufo spottingen en het verband met drugs gebruikt om verschillenden technieken van AI toe tepassen.
wij behandelen de volgenden vragen in dit notebook;

### 1 In hoeverre is er een verband tussen de vorm van de UFO en drugs gebruik per state? (Externe DataSet + KNN Model)

### 2 In hoeverre is er een verband tussen de vorm van de UFO en het land(state) van de waarneming?

### 3 Kunnen we met behulp van clusteren van groepen verband vinden tussen drugs gebruik en de locatie van spottingen (Unsupervised Learning Clustering, interactieve visualisatie)


### 4 Onderzoek met de dataset of Duitsers minder tijd op de website doorbrengen dan gemiddeld (Z-toets)

#### Deze notebook is gemaakt door groep 1; Lucas van de horst, Richard Jansen, Max van Kemenade


### Table of Content

- [Minimale eisen](#Minimale_eisen)
- [Libraries](#libraries)
- [Data cleaning (ufo Set)](#Ufo_clean)
- [Data cleaning (drugs Set)](#Drugs_clean)
- [correlatieonderzoek Ufo Set](#Correl_ufo)
- [Interactieve kaart met Spottingen over tijd](#IntaKaart)
- [Trend van reports over de dag](#Trendtijd)
- [Shapes onderzoek](#shapes)
- [City & States onderzoek](#City_states)
- [Wordcloud](#wordcloud)
- [Inzicht drugs gebruik United States](#Drugs)
- [1 In hoeverre is er een verband tussen de vorm van de UFO en drugs gebruik per state?](#KNN_alg)
- [2 In hoeverre is er een verband tussen de vorm van de UFO en het land(state) van de waarneming?](#ov2)
- [3 Kunnen we met behulp van clusteren van groepen verband vinden tussen drugs gebruik en de locatie van spottingen](#kMEansAlg)
- [4 Onderzoek met de dataset of Duitsers minder tijd op de website doorbrengen dan gemiddeld ](#ztoets)


### Minimale eisen <a id='Minimale_eisen'></a>
Je uitwerking van de data science pipeline moet uiteraard de onderzoeksvragen beantwoorden. Daarnaast moet het minimaal het volgende bevatten:

1. Een uitbreiding van de toegewezen dataset met een externe dataset.

2. Een verkenning van de data met relevante statistische analyses.

3. Een interactieve visualisatie.

4. Een correlatieonderzoek.

5. Toepassing van een supervised machine learning techniek.

6. Toepassing van een unsupervised machine learning techniek.

7. Een hypothesetoets met behulp van de Z-toets.

## libraries <a id='libraries'></a>


In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import holoviews as hv
from sklearn.cluster import KMeans
import plotly

import requests
from PIL import Image
from io import BytesIO
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

hv.extension('bokeh')

## 1# Data verkenning (Data collection)


Hieronder wordt het voorwerk gedaan met de data om een inzicht te krijgen over wat we hebben



In [ ]:
ufo_csv_url = "https://raw.githubusercontent.com/Lucas-vdr-Horst/ufo-observations-ai/master/ufo.csv"
ufo = pd.read_csv(ufo_csv_url, low_memory=False)

## Index verplaatsen
De index wordt hier goed gezet naar de eerste kolom

In [ ]:
ufo.set_index('Unnamed: 0', inplace=True)


## De ingelezen data bekijken en beoordelen

In [ ]:
ufo.head()
print(ufo.dtypes)

In [ ]:
ufo.convert_dtypes().info()

Zoals hierboven bevonden is, moet er een hoop opgeschoond worden

## Missing data controleren (Data cleaning) <a id='Ufo_clean'></a>

In [ ]:
dfCleaned = ufo.copy()

dfCleaned['datetime'] = dfCleaned['datetime'].str.replace('24:00', '00:00')
dfCleaned['datetime']= pd.to_datetime(dfCleaned['datetime'], format="%m/%d/%Y %H:%M") 

dfCleaned['state']  = dfCleaned['state'].fillna('NotUs')

dfCleaned['country']  = dfCleaned['country'].fillna('Missing_Info')

dfCleaned['shape'] = dfCleaned['shape'].fillna('?')

dfCleaned['date posted']= pd.to_datetime(dfCleaned['date posted'], format="%m/%d/%Y") 

dfCleaned['latitude'] = dfCleaned['latitude'].str.replace('33q.200088', '33.200088')
dfCleaned['latitude'] = dfCleaned.latitude.astype(float)

dfCleaned['duration (seconds)'] = dfCleaned['duration (seconds)'].str.replace('`', '') # 2 waardens hebben dit, ez fix
dfCleaned['duration (seconds)']  = dfCleaned['duration (seconds)'].fillna('0') # 2 waardens hebben een nan, liever wil ik drop rows doen
dfCleaned['duration (seconds)'] = dfCleaned['duration (seconds)'].astype(float)

dfCleaned['visit (seconds)'] = dfCleaned['visit (seconds)'].astype(float)

dfCleaned = dfCleaned.drop(columns=['duration (hours/min)', 'comments'])
dfCleaned = dfCleaned[['datetime', 'city','state','country','shape','duration (seconds)','date posted', 'latitude','longitude', 'visit (seconds)']]

dfCleaned = dfCleaned.rename(columns={"duration (seconds)": "duration", "date posted": "date_posted","visit (seconds)": "visit"})

dfCleaned['year_posted']= pd.DatetimeIndex(dfCleaned['date_posted']).year

In [ ]:
dfCleaned.info()

Hieonder hebben wij een overzicht van de verschillende meetniveau's van onze ufo dataset

| variable    | meetniveau |
|-------------|------------|
| datetime    | Continu    |
| city        | Nominaal   |
| state       | Nominaal   |
| country     | Nominaal   |
| shape       | Nominaal   |
| duration    | Continu    |
| comments    | Nominaal   |
| date posted | Continu   |
| latitude    | Continu    |
| longitude   | Continu    |
| visit       | Continu    |

In [ ]:
dfCleaned.isnull().sum()

Alle missing waardens zijn weggewerkt in deze dataset

## Data cleaning (drugs Set) <a id='Drugs_clean'></a>

Source van externe dataset
https://pdas.samhsa.gov/saes/state#age-chart-tab

In [ ]:
marijuana_csv_url = "https://raw.githubusercontent.com/Lucas-vdr-Horst/ufo-observations-ai/master/MarijuanaUS.csv"
df2 = pd.read_csv(marijuana_csv_url)

In [ ]:
df2.head()

In [ ]:
df2.convert_dtypes().info()

In [ ]:
df2.isnull().sum()

In [ ]:
df2

## Clean drugs database voor merging

In [ ]:
df2.drop(columns=['outcome', 'age_group'])

df2['year_pair'] = df2['year_pair'].astype(str).str[0:4]
df2['year_pair'] = df2['year_pair'].astype(int)

states = {'al':'Alabama','ak':'Alaska', 'az':'Arizona', 'ar':'Arkansas', 'ca':'California',
       'co':'Colorado', 'ct':'Connecticut', 'de':'Delaware', 'dc':'District of Columbia',
       'fl':'Florida', 'ga':'Georgia', 'hi':'Hawaii', 'id':'Idaho', 'il':'Illinois', 'in':'Indiana',
       'ia':'Iowa', 'ks':'Kansas', 'ky':'Kentucky', 'la':'Louisiana', 'me':'Maine', 'md':'Maryland',
       'ma':'Massachusetts', 'mi':'Michigan', 'mn':'Minnesota', 'ms':'Mississippi',
       'mo':'Missouri', 'mt':'Montana', 'ne':'Nebraska', 'nv':'Nevada', 'nh':'New Hampshire',
       'nj':'New Jersey', 'nm':'New Mexico', 'ny':'New York', 'nc':'North Carolina',
       'nd':'North Dakota', 'oh':'Ohio', 'ok':'Oklahoma', 'or':'Oregon', 'pa':'Pennsylvania',
       'ri':'Rhode Island', 'sc':'South Carolina', 'sd':'South Dakota', 'tn':'Tennessee',
       'tx':'Texas', 'ut':'Utah', 'vt':'Vermont', 'va':'Virginia', 'wa':'Washington',
       'wv':'West Virginia', 'wi':'Wisconsin', 'wy':'Wyoming'}
for short, state in states.items():
    df2['state'] = df2['state'].str.replace(state, short)

Hier onder staat de postal code in verband met de staat

|  State/District  | Postal Code |
|-------------|------------|
| Alabama	    | 	AL
| Alaska	| 	AK
| Arizona	| 	AZ
| Arkansas	| 	AR
| California	| 	CA
| Colorado	| 	CO
| Connecticut	| 	CT
| Delaware	| 	DE
| District of Columbia| 	DC
| Florida	| 	FL
| Georgia	| 	GA
| Hawaii	| 	HI
| Idaho	| 	ID
Illinois	| 	IL
Indiana	| 	IN
Iowa	| 	IA
Kansas	| 	KS
Kentucky	| 	KY
Louisiana	| 	LA
Maine	| 	ME
Maryland	| 	MD
Massachusetts	| 	MA
Michigan	| 	MI
Minnesota	| 	MN
Mississippi	| 	MS
Missouri	| 	MO
Montana	| 	MT
Nebraska	| 	NE
Nevada	| 	NV
New Hampshire	| 	NH
New Jersey	| 	NJ
New Mexico	| 	NM
New York	| 	NY
North Carolina	| 	NC
North Dakota | 	ND
Ohio	| 	OH
Oklahoma	| 	OK
Oregon	| 	OR
Pennsylvania	| PA
Rhode Island	| 	RI
South Carolina	| 	SC
South Dakota	| 	SD
Tennessee	| 	TN
Texas	| 	TX
Utah	| 	UT
Vermont	| 	VT
Virginia	| 	VA
Washington	| 	WA
West Virginia	| 	WV
Wisconsin	| 	WI
Wyoming	| 	WY

## Mergen drugs database with ufo database

In [ ]:
dfMerge = pd.merge(dfCleaned, df2,  how='left', left_on=['state','year_posted'], right_on = ['state','year_pair'])
dfMerge = dfMerge.dropna()
dfMerge = dfMerge.rename(columns={"year_pair": "year"})
dfMerge = dfMerge.drop('year_posted', axis=1)
dfMerge = dfMerge[dfMerge.country == "us"]  #drugs database is based america only.
dfMerge

In [ ]:
dfMerge.isnull().sum()

## Correlatie onderzoek <a id='Correl_ufo'></a>

In [ ]:
pd.plotting.scatter_matrix(dfCleaned[['duration', 'latitude','longitude','visit']])
plt.show()

In [ ]:
dfCleaned.corr()

Hierboven staan de uitkomsten van het corrolatie onderzoek. Hiervoor is er onderzocht of er een verband, corrolatie, valt te vinden tussen de kwantitatieve waardes uit de ufo database. Om dit zichtbaar te maken is er een matrix scatterplot gemaakt. Alleen hieruit was geen duidelijk verband zichtbaar. Dit blijkt later ook grooten deels uit het corrolatie onderzoek met de corr functie van de pandas libary. Bijna elk verband is ruim onder de 0.3 wat betekend dat er een slecht verband is. Het enige matige verband is tussen longitude en latitude, maar hiervan was al bekent dat er een verband was aangezien het gaat om de x en y as van de planeet.

## Interactieve wereld kaart met spottingen <a id='IntaKaart'></a>

In [ ]:
min_date, max_date = dfCleaned['datetime'].min(), dfCleaned['datetime'].max()
amount_steps = 20
step_size = (max_date - min_date) / amount_steps

def step_to_timerange(step):
  return min_date + step_size*step, min_date + step_size*(step+1)

def time_slice(start_time, end_time):
  mask = (dfCleaned['datetime'] > start_time) & (dfCleaned['datetime'] <= end_time)
  return hv.Scatter(zip(dfCleaned.loc[mask]['longitude'], dfCleaned.loc[mask]['latitude']), label='Spottingen vanaf jaar')

scatter_dict = {step_to_timerange(i)[0].year:time_slice(*step_to_timerange(i)) for i in range(amount_steps)}

hv.extension('bokeh')
world_plot = hv.HoloMap(scatter_dict)
world_plot.opts(width=600, height=300)

Hier zijn de spottingen-locaties gesliced in tijd-periodes, met de slider kan je "door te tijd reizen"

## Trend voor de tijd <a id='Trendtijd'></a>

In [ ]:
date = np.array(ufo['datetime'])
dateEdit = np.array([])
count =0
for i in date:
    date[count] =  date[count][-5:]
    count +=1

df = np.unique(date, return_counts=True)
x= df[0]
y= df[1]
fig, ax1 = plt.subplots()
ax1.plot(x,y)

dateEdit = date[:100]
plt.title('Tijden gespot in minuten')
plt.show()

dfCleaned['only_time'] = dfCleaned['datetime'].dt.hour
plt.hist(dfCleaned['only_time'])
plt.title('Tijden gespot in uren')
plt.show()

Er zijn aanzienlijk meer piekjes op hele uren en halve uren bij de eerste plot, en bij de tweede zien we dat er meer spottingen s' nachts zijn.

## Shapes onderzoek <a id='shapes'></a>

In [ ]:
dfCleaned['shape'].unique()

In [ ]:
dfCleaned['shape'].value_counts()

## City & States onderzoek <a id='City_states'></a>

In [ ]:
dfCleaned['country'].unique()

In [ ]:
dfCleaned['country'].describe()

De top 50 van spotlocaties

In [ ]:
city = dfCleaned['city'].value_counts().index
city_count = dfCleaned['city'].value_counts().values

plt.subplots(figsize=(24,8))
sns.barplot(city[:50], city_count[:50])
plt.xticks(rotation=90, fontsize=16)
plt.yticks(fontsize=20)
plt.title('De top 50 van spotlocaties')
plt.show()

In [ ]:
print('Total:          '+str(dfCleaned['country'].count()))
print(dfCleaned['country'].value_counts())

Plaats van meeste spottingen in de US

In [ ]:
states_us = dfCleaned[dfCleaned.country == "us"]["state"].value_counts().index
states_ratio = dfCleaned[dfCleaned.country == "us"]["state"].value_counts().values

plt.subplots(figsize=(24,8))
sns.barplot(states_us, states_ratio)
plt.xticks(rotation=45, fontsize=16)
plt.yticks(fontsize=20)
plt.title('Plaats van meeste spottingen in de US')
plt.show()

Plaats van meeste spottingen in Canada

In [ ]:
canada = dfCleaned[dfCleaned.country == "ca"]['city'].value_counts().index
canada_count = dfCleaned[dfCleaned.country == "ca"]['city'].value_counts().values

plt.subplots(figsize=(24,8))
sns.barplot(canada[:50], canada_count[:50])
plt.xticks(rotation=90, fontsize=16)
plt.yticks(fontsize=20)
plt.title('Plaats van meeste spottingen in Canada')
plt.show()

Plaats van meeste spottingen in Engeland

In [ ]:
england = dfCleaned[dfCleaned.country == "gb"]['city'].value_counts().index
england_count = dfCleaned[dfCleaned.country == "gb"]['city'].value_counts().values

plt.subplots(figsize=(24,8))
sns.barplot(england[:50], england_count[:50])
plt.xticks(rotation=90, fontsize=16)
plt.yticks(fontsize=20)
plt.title('Plaats van meeste spottingen in Engeland')
plt.show()

Plaats van meeste spottingen in Australië 

In [ ]:
australia = dfCleaned[dfCleaned.country == "au"]['city'].value_counts().index
australia_count = dfCleaned[dfCleaned.country == "au"]['city'].value_counts().values

plt.subplots(figsize=(24,8))
sns.barplot(australia[:50], australia_count[:50])
plt.xticks(rotation=90, fontsize=16)
plt.yticks(fontsize=20)
plt.title('Plaats van meeste spottingen in Australië ')
plt.show()

Plaats van meeste spottingen in Duitsland

In [ ]:
germany = dfCleaned[dfCleaned.country == "de"]['city'].value_counts().index
germany_count = dfCleaned[dfCleaned.country == "de"]['city'].value_counts().values

plt.subplots(figsize=(24,8))
sns.barplot(germany[:50], germany_count[:50])
plt.xticks(rotation=90, fontsize=16)
plt.yticks(fontsize=20)

plt.show()

## Wordcloud <a id='wordcloud'></a>

In [ ]:
image_url = "https://raw.githubusercontent.com/Gommy111/ufo-observations-ai/master/ufo.png"
ufo_mask = np.array(Image.open(BytesIO(requests.get(image_url).content)))
ufo_mask[ufo_mask < 200] = 0
ufo_mask[ufo_mask >= 200] = 255

In [ ]:
text = "".join(ufo['comments'].astype(str))
wordcloud = WordCloud(
    stopwords=STOPWORDS,
    max_words=200,
    background_color="white",
    mask=ufo_mask,
    contour_width=2,
    contour_color='firebrick'
).generate(text)

plt.figure(figsize=(15, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Wordcloud of the comments")
plt.show()

##Inzicht drugs gebruik United States  <a id='Drugs'></a>

In [ ]:
df2

In [ ]:
plt.subplots(figsize=(24,8))
sns.lineplot(data=df2, x='year_pair', y='estimate', hue='state')
plt.legend(loc='upper left')
#sns.scatterplot(data=df2, x="year", y='estimate')
sns.relplot(data=df2, x="year_pair", y='estimate', kind="line")
#sns.lineplot(data=df2, x="year", y='estimate', hue="state")
#sns.scatterplot(data=statestdf2, x="year", y='estimate',hue="state")
#sns.scatterplot(data=stateCalstdf2, x="year", y='estimate',hue="state")
plt.xticks(rotation=45, fontsize=16)
plt.yticks(fontsize=20)

Een conclusie die we uit deze simpele visualisatie kunnen halen is dat het drugs gebruik is toegenomen door de jaren heen

# Onderzoeksvragen:

## 1 In hoeverre is er een verband tussen de vorm van de UFO en drugs gebruik per state? 
Supervised learning ov 1 KNN Model <a id='KNN_alg'></a>


Hier passen wij het KNN algoritme toe om te voorspellen welke vorm de ufo heeft

**Warning de code bij onderzoeksvraag 1 kan lang duren**

In [ ]:
from sklearn import datasets                               # Bevat de Iris-dataset
from sklearn.model_selection import train_test_split       # Opsplitsen Train/Test
from sklearn.linear_model import LinearRegression          # Lineaire regressie
from sklearn.cluster import KMeans                         # K-Means clustering
from sklearn.neighbors import KNeighborsClassifier         # K-NearestNeighbors classifier
from sklearn.metrics import accuracy_score                 # Meassuring accuracy
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
dfKNN = dfMerge.copy()
dfKNN = dfKNN.drop(columns=['city','state','country','outcome', 'age_group'])
dfKNN.dtypes

Normalisatie datetime en 'shapes' naar voren verplaatsen

In [ ]:
dfKNN['datetime'] = (dfKNN['datetime'] - dfKNN['datetime'].min())  / np.timedelta64(1,'D')
dfKNN['date_posted'] = (dfKNN['date_posted'] - dfKNN['date_posted'].min())  / np.timedelta64(1,'D')

col = dfKNN.pop("shape")

x = dfKNN.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x[1:])
df = pd.DataFrame(x_scaled)

dfKNN.insert(0, col.name, col)

df.describe()

In [ ]:
dfKNN.dtypes

In [ ]:
#baseline als je cilinder zou gokken
f"{17872/88679*100}%"

In [ ]:
# deze code maakt een lijst met alle combinaties mogelijk zonder duplicates van de meegegeven lisjt
import copy
def combinations(target,data):
    
    for i in range(len(data)):
        new_target = copy.copy(target)
        new_data = copy.copy(data)
        new_target.append(data[i])
        new_data = data[i+1:]
        print(new_target)
        combolst.append(new_target)
        combinations(new_target,new_data)
                    
target = []
data = list(dfKNN.columns)
combolst = []
combinations(target,data[1:])

De code hier onder bepaald met een script de best mogelijk uitkomst (duur alleen lang om alles te testen)

In [ ]:
sampleDf = dfKNN.sample(n=5000, random_state=0)
scoreFirm = 0.0
#i =20
for i in range(3000,5100,100):
    for dt in combolst:
        #print(dt)
        x = sampleDf[dt]
        y = sampleDf[['shape']]

        x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

        knn = KNeighborsClassifier()

        knn.set_params(n_neighbors = i, weights = 'distance', algorithm = 'brute',leaf_size = 30)

        knn.fit(x_train,np.ravel(y_train))

        y_pred = knn.predict(x_test)
        accuracy_score(y_test, y_pred)

        score = knn.score(x_test,y_test)
        #Deze print statements zijn handig als je alles wilt inzien
        #print(knn.score(x_test,y_test))
        #print(dt)
        #print("['duration', 'latitude', 'longitude', 'visit']")
        #print("k =", i)
        if score > scoreFirm:
            scoreFirm = score
            print(score)
            print(dt)
            print("k =", i)
            print("Highest score from botom")
            print("")

De code hieronder laat de best mogelijke uitkomst door ons gevonden weergeven waarmee we alleen kijken naar de drugs

In [ ]:
x = dfKNN[['estimate', 'ci_lower', 'ci_upper']]
y = dfKNN[['shape']]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

knn = KNeighborsClassifier()

knn.set_params(n_neighbors = 4000, weights = 'distance', algorithm = 'brute',leaf_size = 30)

knn.fit(x_train,np.ravel(y_train))

y_pred = knn.predict(x_test)
accuracy_score(y_test, y_pred)

score = knn.score(x_test,y_test)
print(knn.score(x_test,y_test))
print("['estimate', 'ci_lower', 'ci_upper']")
print("k =", i)

De code hier onder laat de best gevonden parameters voor het algortime zien

In [ ]:
x = dfKNN[['datetime', 'duration', 'date_posted', 'latitude', 'longitude']]
y = dfKNN[['shape']]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

knn = KNeighborsClassifier()

knn.set_params(n_neighbors = 4000, weights = 'distance', algorithm = 'brute',leaf_size = 30)

knn.fit(x_train,np.ravel(y_train))

y_pred = knn.predict(x_test)
accuracy_score(y_test, y_pred)

score = knn.score(x_test,y_test)
print(knn.score(x_test,y_test))
print("['datetime', 'duration', 'date_posted', 'latitude', 'longitude']")
print("k =", i)

Conclusie

Na een langetijd van parameters proberen en n factor kunnen we geen verband vinden op de voorspelling van de vorm in verhouding met drugs gebruik. Wel op merkelijk is dat het algoritme op de drugs collumns nog best hoog scoort

## 2 In hoeverre is er een verband tussen de vorm van de UFO en het land(state) van de waarneming?
<a id='ov2'></a>

In [ ]:
lst = dfMerge.state.unique() # Alleen amerikaanse staten.
result = []
# lst.sort(reverse=True)
for i in lst:
    state = dfMerge[['state', 'shape']][dfMerge.state == i]
    h = state['shape'].value_counts()[:1]
    result.append((i, h))

print(result)

Hierboven kun je zien dat de 'shape' licht(light) het meeste voor komt in alle state behalve North Dakota en District of Columbia. Alleen hieruit blijkt dat er nog geen verband is.

In [ ]:
lst = dfCleaned.country.unique() # de landen
result = []
print(lst)
# lst.sort(reverse=True)
for i in lst:
    state = dfCleaned[['state', 'shape']][dfCleaned.country == i]
    h = state['shape'].value_counts()[:1]
    result.append((i, h))

print(result)

Hierboven geldt eigenlijk netzoals bij de amerikaanse states dat licht het meest voorkomend is, maar dat er niet echt een verband is.

**Warning kan lang duren**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le  = LabelEncoder()

le.fit(dfMerge['state'])
dfMerge['state_int'] = le.transform(dfMerge['state'])

le.fit(dfMerge['shape'])
dfMerge['shape_int'] = le.transform(dfMerge['shape'])

sns.pairplot(data = dfMerge)

In [ ]:
dfMerge[['shape_int','state_int']].corr()

Tot slot is er nog een corrolatie onderzoek gedaan. Hiervoor zijn de states en shapes geïndexeert en ook hieruit blijkt dat er geen verband is aangezien de corrolatie ruim onder de 0.3 zit, wat betekened dat er een verband tussen zit.

Hieruit kan dus geconcludeert worden dat er geen verband bestaat tussen de staat en shape. Wel valt het op de ufo's meer worden geïdentificeerd met licht dan met een schotel form.

## 3 Kunnen we met behulp van clusteren van groepen verband vinden tussen drugs gebruik en de locatie van spottingen 
UnsupervisedSupervised learning ov 3 K Means Clusteren <a id='kMEansAlg'></a>

In [ ]:
plotly.offline.init_notebook_mode()     # Als je een offline notebook gebruikt, uncomment deze dan
#plotly.io.renderers.default = 'colab'  # Als je Google Colab gebruikt, uncomment deze dan

Eerst nemen we een stukje van de dataset, dat scheelt heel erg veel processing.

En dan normalizeren we de latitude, longitude en estimate die we dan weer terug stoppen in de sample

In [ ]:
sample = dfMerge.sample(n=1000, random_state=1)
to_normalize = sample[['latitude', 'longitude', 'estimate']]
#normalized_df = (to_normalize-to_normalize.mean())/to_normalize.std()
normalized_df = (lambda x: (x-x.min())/(x.max()-x.min()))(to_normalize)
sample[['latitude', 'longitude', 'estimate']] = normalized_df
sample.describe()

Hier maken we het KMeans model aan met de hyperparaneter 7 clusters.

KMeans wil het in zo'n formaat: [(a, b, c)] dus we zippen de latitude, longitude en estimate samen.

En dan trainen we hem op de data.

In [ ]:
cluster_model = KMeans(n_clusters=3, random_state=1)
zipped = list(zip(sample['latitude'], sample['longitude'], sample['estimate']))
cluster_model.fit(np.array(zipped))

Omdat het 3 input variablen zijn die gegroepeerd zijn, kunnen we hier een 3D plot voor gebruiken.

Eerst hebben we een functie die onze dataframe kan omzetten naar een formaat dat plotly wil, en daarna hebben we de layout voor de plot.

In [ ]:
def create_plotable_3d_data(df, cluster_column_name: str):
    unique_cluster_names = df[cluster_column_name].unique()
    result_list = []
    
    for cluster_name in unique_cluster_names:
        cluster = df.loc[df[cluster_column_name] == cluster_name]
        result_list.append({
            'mode': 'markers',
            'name': str(cluster_name),
            'type': 'scatter3d',
            'x': cluster.values[:, 0], 'y': cluster.values[:, 1], 'z': cluster.values[:, 2],
            'marker': {'size': 2}
        })
        result_list.append({
            'alphahull': 5,
            'name': str(cluster_name),
            'opacity': 0.1,
            'type': 'mesh3d',
            'x': cluster.values[:, 0], 'y': cluster.values[:, 1], 'z': cluster.values[:, 2],
            'showscale': True
        })
    
    return result_list

layout = dict(
    title = 'Interactieve 3D visualisatie',
    scene = dict(
        xaxis = dict( zeroline=True ), xaxis_title = 'latitude',
        yaxis = dict( zeroline=True ), yaxis_title = 'longitude',
        zaxis = dict( zeroline=True ), zaxis_title = 'estimate'
    )
)

Hieronder is een demo'tje waar het gegroepeerd is op de vorm van de ufo.

In [ ]:
clustering_df = sample[['latitude', 'longitude', 'estimate']]
clustering_df = clustering_df.copy()
clustering_df['y'] = sample['shape']
plotly.offline.iplot({'data': create_plotable_3d_data(clustering_df, 'y'), 'layout': layout})

Maar hier is ie werkelijk geclustered aan de hand van onze KMeans model.

In [ ]:
clustering_df = sample[['latitude', 'longitude', 'estimate']].copy()
clustering_df['y'] = cluster_model.predict(np.array(zipped))
plotly.offline.iplot({'data': create_plotable_3d_data(clustering_df, 'y'), 'layout': layout})

Er komen drie klusters uit, één hoog drugs gebruik cluster per kust en één cluster in het zuiden waar veel minder drugs gebruik is.

Dus om de vraag te beantwoorden:

Er zit een verband tussen de locatie van de spottingen en het drugs gebruik. We zouden graag willen dat we een interresantere conclusie konden trekken maar meer vinden we niet.

Misschien heeft verband ook nog wel te maken met het noordelijke buurland waar marijuana gelegaliseerd is. Maar dat verlaat het UFO onderwerp.

---

Sidenote: als je de plot onder de juiste hoek bekijkt zie je dat het de vorm van een **U** heeft... van **UFO**

In [ ]:
from IPython.display import HTML
HTML('<img src="https://media1.tenor.com/images/10e0e40ab4fb697ea64687ee57b181b8/tenor.gif?itemid=5096707">')

## 4 Onderzoek met de dataset of Duitsers minder tijd op de website doorbrengen dan gemiddeld
Z-toets ov 4 <a id='ztoets'></a>

Canvas Text

UFO waarnemingen

In een artikel in een Duits dagblad werd onlangs gesteld dat Duitsers met interesse in het bovenmenselijke ondergemiddeld actief zijn op internet (ten opzichte van andere nationaliteiten), omdat er een grote UFO-vereniging bestaat die de noodzaak voor online delen van informatie wegneemt.

Onderzoek met de dataset of Duitsers minder tijd op de website doorbrengen dan gemiddeld (zie als populatie de dataset zonder de Duitse waarnemingen). Neem als betrouwbaarheid 90%.

hypothesen op stellen

H0: er is geen verschil ($\mu_{Duitsers vistit tijd} = \mu_{populatie}$)

H1: er is een verschil ($\mu_{Duitsers vistit tijd} \le \mu_{populatie}$)

Er is sprake van een _rechtszijdige_ toets.

In [ ]:
germanystd = dfCleaned[dfCleaned.country == "de"]['visit'].std()
germanystd

In [ ]:
import scipy.stats as stats
import scipy as sp

germanyCount = len(dfCleaned[dfCleaned.country == "de"]['visit'])
germanySum = dfCleaned[dfCleaned.country == "de"]['visit'].sum()
germanyMean = dfCleaned[dfCleaned.country == "de"]['visit'].mean()
germanystd = dfCleaned[dfCleaned.country == "de"]['visit'].std()

allPopMean = dfCleaned[dfCleaned.country != "de"]['visit'].mean() 
allPopcount = len(dfCleaned[dfCleaned.country != "de"]['visit'])

n = germanyCount  #get coutry DE count
print("Steekproefgrootte n = {}".format(n))

x_ = germanyMean  #get gem visit all country
print("Steekproefgemiddelde x = {:.2f}".format(x_))

s = germanystd  #standaard diviatie van Duitsland visit tijd
print("Standaardafwijking steekproef s = {:.2f}".format(s))


mu = allPopMean  #Alle country BEHALVE Duitsland
print("gemiddelde populatie mu = {:.4f}".format(mu))

# Standaardfout
se = s/np.sqrt(n)
print("Standaardfout se = {:.2f}".format(se))

# Bereken grens van de z-waarde bij alpha
alpha = 0.05 #dit is 90% betrouwbaarheid (als goed is)

z_grens = stats.norm.ppf((1-alpha) )
print("De grens z_waarde is {:.2f}".format(z_grens))
grens = z_grens * se + mu
print("De grenswaarde is {:.2f}".format(grens))



#plotten van een schets docent code
# Schets de verdeling 
x = np.linspace(230,270,100)
nv = stats.norm(loc=mu, scale=s/np.sqrt(n))   
y = nv.pdf(x)

plt.figure(figsize=(12,6), dpi=80)
plt.plot(x,y)
plt.fill_between(x, y, where=(x>grens))

# plaats van het steekproefgemiddelde
plt.plot([x_,x_],[0,0.1])
plt.show()


# Bereken z-waarde
z = (x_ - mu)/se
print("De z-waarde (rechts) z = {:.2f}".format(z))

# Bereken p-waarde
p = 1-stats.norm.cdf(z)
print("De p-waarde p = {:.4f}".format(p))

Conclusie

Er geldt dat  p = 0.8727 > α=0.05 . Dat p = 0.8727 betekent dat deze uitkomst door toeval kan plaatsvinden, dus we verwerpen  H0  niet: er is dus geen significant verschil in HT

